# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Create variable for the file path.
file = "output_data/cities.csv"
#Create dataframe with CSV fle 
cities_df = pd.read_csv(file)
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,63.98,91,5,3.53,ZA,1612489893
1,1,ponta do sol,32.6667,-17.1000,48.97,67,83,1.34,PT,1612490167
2,2,flinders,-34.5833,150.8552,83.98,70,13,3.13,AU,1612490226
3,3,asau,46.4333,26.4000,37.38,87,40,3.09,RO,1612490557
4,4,huanren,41.2647,125.3667,23.18,93,91,3.22,CN,1612490557


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity_rate = cities_df["Humidity"].astype(float)

# Create a poverty Heatmap layer
#create the canvas for the map
#zoom level is 0-21 (0 is more zoomed out), center needs to a lat/long
fig = gmaps.figure(zoom_level=3, center=(0, 0))

#creating the heatmap
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=max(cities_df["Humidity"]),
                                 point_radius = 3)
#print heatmap layer on top of canvas
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
#CONDITIONS
#A max temperature lower than 80 degrees but higher than 75.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

#filter by ideal conditions
conditions = cities_df.loc[(cities_df["Max Temp"]< 80)&(cities_df["Max Temp"]>75) & (cities_df["Wind Speed"]<10) & (cities_df["Cloudiness"] == 0)]
#drop any rows that don't meet conditions. 
conditions



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,minna,9.6139,6.5569,78.78,18,0,4.12,NG,1612490560
40,40,sao joao da barra,-21.6403,-41.0511,77.47,82,0,8.96,BR,1612490092
48,48,touros,-5.1989,-35.4608,78.78,78,0,4.63,BR,1612490160
54,54,pyay,18.8167,95.2167,75.36,40,0,2.06,MM,1612490563
90,90,natal,-5.7950,-35.2094,78.78,78,0,3.60,BR,1612490568
225,225,axim,4.8699,-2.2405,79.16,88,0,3.01,GH,1612490594
417,417,lokoja,7.8024,6.7430,76.10,21,0,0.93,NG,1612490625
547,547,san rafael,-34.6177,-68.3301,75.36,26,0,3.07,AR,1612490538


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# find the closest hotel of each type to coordinates
hotels_df = conditions

#Create parameters for gmaps iteration 
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotels_df.iterrows():
    # get lat and lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location based on each lat/lng from dataframe 
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #define city variable
    city = row["City"]
    
    # create url and make API request
    print(f"Results for {'city'}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #cities that have a hotel meeting the parameters defined will print the closest hotel
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotels_df.loc[index, 'Hotel Name'] = results[0]['name']
    #cities that do not have a hotel meeting the parameters will indicate this and skip.
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Results for city: minna.
Closest hotel is Royal Park hotel & suites.
------------
Results for city: sao joao da barra.
Closest hotel is Pousada Porto De Canoas.
------------
Results for city: touros.
Closest hotel is INN NEW HORIZON.
------------
Results for city: pyay.
Closest hotel is Shwe Pyi Thar.
------------
Results for city: natal.
Closest hotel is Villa Park Hotel | Natal.
------------
Results for city: axim.
Closest hotel is AXIM BEACH RESORT AND SPA.
------------
Results for city: lokoja.
Closest hotel is Hotel Charvid.
------------
Results for city: san rafael.
Closest hotel is Hotel San Rafael.
------------


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
hotel_locations = hotels_df[["Lat", "Lng"]]

In [47]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))